In [ ]:
from __future__ import absolute_import, division, print_function
from copy import deepcopy
from matplotlib import pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split

import os
import random
import json
import math
import numpy as np
import pandas as pd
import cv2
import time
import funcy
from itertools import groupby
from operator import itemgetter

import detectron2
import detectron2.data.datasets
import detectron2.data.catalog
import detectron2.utils.logger
import detectron2.utils.visualizer
import detectron2.config
import detectron2.engine
from detectron2.engine import DefaultPredictor
import detectron2.evaluation
import detectron2.model_zoo

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
import detectron2.evaluation
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


In [ ]:
# magic to get in-line figures to display
%matplotlib inline
plt.rcParams['figure.figsize'] = [20, 10]

# magic to get tensorboard to display
#%load_ext tensorboard # I commented this
%reload_ext tensorboard

In [ ]:
cfg = detectron2.config.get_cfg()
cfg.merge_from_file(detectron2.model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")) 
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.75 # Set threshold for this model, different thresholds will be apllied below
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0 # non-maximum suppression 
cfg.MODEL.WEIGHTS = "New_Nick_Failure_Weights.pth" # Loads the trained weights of nick failure defect
#cfg.MODEL.WEIGHTS = "New_chipped_tooth_weights.pth"  # Loads the trained weights of chipped tooth defect
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 # number of foreground classes
cfg.DATASETS.TEST = ("test", ) # assign Test set
predictor = detectron2.engine.DefaultPredictor(cfg) # Predicts the instances on images

In [ ]:
def count_instances(directory): 
    images_directory = glob (directory)
    predictions_dict = {}
    ground_truth_dict = {}
    annotations_array = []
    images_names_list = []
    camera_defects = []
    predictions_numbers_list = []
    defected_images = []
    final_defected_images = []
    instances_number = 0
    for image in range(len(images_directory)):
        inspected_image = cv2.imread(images_directory[image])
        prediction = predictor(inspected_image)
        #visualizer_predictions_inspected = detectron2.utils.visualizer.Visualizer(inspected_image[:, :, ::-1], metadata_test, scale=1)
        visualizer_predictions_inspected = detectron2.utils.visualizer.Visualizer(inspected_image[:, :, ::-1], None, scale=1)
        vis_predictions_inspected = visualizer_predictions_inspected.draw_instance_predictions(prediction["instances"].to("cpu"))
        #plt.imshow(vis_predictions_inspected.get_image()) # To visualize the inspected image with instances
        #plt.show()
        image_name = os.path.basename(images_directory[image])       
        split_image_name = image_name.split('_')
        images_names_list.append(split_image_name[0])
        
        ground_dict= {
          image_name:annotations_array
        }
        
        ground_truth_dict.update(ground_dict) 
        
        with open("ground_truth_negatives.json", "w") as outfile: 
            json.dump(ground_truth_dict, outfile)


        predictions_boxes = prediction["instances"].get('pred_boxes')
        predictions_scores = prediction["instances"].get('scores')
        boxes_array=[]
        scores_array=[]

        # Getting bboxes and scores for predicitons
        instances_number = instances_number + len(predictions_boxes)
        predictions_numbers_list.append(len(predictions_boxes))       
        res = {val : predictions_numbers_list[idx] for idx, val in enumerate(images_names_list)} 
        
        predictions_numbers_dict= pd.DataFrame({
          'gear_number':images_names_list,
          "predictions_number": predictions_numbers_list,

            })
        
        if len(predictions_boxes) >0 :
            final_defected_images.append(image_name)
            defected_images.append(split_image_name[0])
            camera_defects.append(split_image_name[2]) #to get the number of the camera which has the defected image
        for predict in range(len(predictions_boxes)):

            coordinates = [round(float(list(predictions_boxes)[predict][0]),4),round(float(list(predictions_boxes)[predict][1]),4),round(float(list(predictions_boxes)[predict][2]),4),round(float(list(predictions_boxes)[predict][3]),4)]

            scores = round(float(predictions_scores[predict]),4)
           
            boxes_array.append(coordinates)
            scores_array.append(scores)
   
        pred_dict= {
          os.path.basename(image_name):{
          "boxes": boxes_array,
          "scores": scores_array
        }}
        
        predictions_dict.update(pred_dict) 
        
        with open("predictions_positives.json", "w") as outfile: 
            json.dump(predictions_dict, outfile) 

    return instances_number , final_defected_images, predictions_dict

In [ ]:
 def find_consecutive(image_name_array, num_total_images, min_num_consecutive = 2):
        
        num_consecutive = 0

        # parse camera and image number from file name
        cam_0 = []
        cam_1 = []
        for image_name in image_name_array:
            x = image_name.split(".")
            y = x[len(x)-2]
            z = y.split("_")
            cam_num = int(z[len(z)-2])
            img_num = int(z[len(z)-1])
            if cam_num == 0: 
                cam_0.append(int(img_num))
            else:       
                cam_1.append(int(img_num))

        if num_total_images == 44:
            if (0 in cam_0) & (21 in cam_0) :
                cam_0.append(-1)
            elif (0 in cam_1) & (21 in cam_1) :
                cam_1.append(-1)
        elif num_total_images == 52:
            if (0 in cam_0) & (25 in cam_0) :
                cam_0.append(-1)
            elif (0 in cam_1) & (25 in cam_1) :
                cam_1.append(-1)         
        
            
        cam_0 = sorted(cam_0)
        cam_1 = sorted(cam_1)

        for k, g in groupby(enumerate(cam_0), lambda ix : ix[0] - ix[1]):
            if (len(list(map(itemgetter(1), g))) >= min_num_consecutive): 
                num_consecutive = num_consecutive + 1

        for k, g in groupby(enumerate(cam_1), lambda ix : ix[0] - ix[1]):
            if (len(list(map(itemgetter(1), g))) >= min_num_consecutive):
                num_consecutive = num_consecutive + 1


        return num_consecutive


## Run the below cell to apply the number of defective images constrain

In [ ]:
defected_gears_number = []
inspected_gears = []
thresholds_predictor = [.75, .8, .85, .90, .95]
#thresholds_predictor = [.9]


#folders = glob ('All_Perfect_Gears/*') # to inspect the non-defective gears with the trained model of either nick failure or chipped tooth
folders = glob ('Defected_Nick_Failure_30_gears_all_images/*') # to inspect the 30 benchmark defective gears of nick failure
#folders = glob ('Defected_Damaged_teeth_30_gears_all_images/*') # to inspect the 30 benchmark defective gears of chipped tooth
for threshold in thresholds_predictor:
    print('Threshold is', threshold)
    for fold in folders:
        inspected_gears.append(fold)
        directory = (fold+"/*")    
        gear_number = directory.split('/')[1]
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
        predictor = detectron2.engine.DefaultPredictor(cfg)
        output = count_instances(directory)
        defective_images_number = output[1]
        if len(defective_images_number) >= 4: # change this number from 1 to 4 (threshold of number of defective images) 
            defected_gears_number.append(gear_number)
    print('Number of inspected gears',len(inspected_gears))            
    print('Number of defected gears',len(defected_gears_number))

## Run the below cell to apply the number of consecutive defective images constrain

In [ ]:
defected_gears_number = []
inspected_gears = []
thresholds_predictor = [.75, .8, .85, .90, .95]
#thresholds_predictor = [.95]

#folders = glob ('All_Perfect_Gears/*')
folders = glob ('Defected_Nick_Failure_30_gears_all_images/*')
#folders = glob ('Defected_Damaged_teeth_30_gears_all_images/*')

for threshold in thresholds_predictor:
    print('Threshold',threshold)
    
    for fold in folders:
        inspected_gears.append(fold)
        directory = (fold+"/*")    
        gear_number = directory.split('/')[1]
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
        predictor = detectron2.engine.DefaultPredictor(cfg)
        num_detections, defect_image_names, pred_dict = count_instances(directory)
        num_consecutive =find_consecutive(defect_image_names,len(glob(directory)) , 4) # change the number from (2 to 4)for testing with different number of consecutive images
        if num_consecutive > 0 :
            defected_gears_number.append(gear_number)
    print('Number of inspected gears',len(inspected_gears))            
    print('Number of defected gears',len(defected_gears_number))
